<a href="https://colab.research.google.com/github/1bharadvaja/ML-Code/blob/master/CUDA_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!pip install --quiet "nvcc4jupyter==1.1.0"


In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpdfyy0gr7".


In [ ]:
%%writefile hello.cu
#include <stdio.h>
#include <cuda_runtime.h>

#define CHECK(call) do {                                           \
  cudaError_t err__ = (call);                                      \
  if (err__ != cudaSuccess) {                                      \
    fprintf(stderr, "CUDA error %s at %s:%d\n",                    \
            cudaGetErrorString(err__), __FILE__, __LINE__);        \
    return 1;                                                      \
  }                                                                \
} while (0)

__global__ void hello(){
  printf("Hello from block %u, thread %u\n", blockIdx.x, threadIdx.x);
}

int main(){
  int devCount = 0;
  CHECK(cudaGetDeviceCount(&devCount));
  if (devCount == 0) { fprintf(stderr, "No CUDA device visible.\n"); return 1; }

  // Optional: make the device printf buffer large (not needed here, but good habit)
  CHECK(cudaDeviceSetLimit(cudaLimitPrintfFifoSize, 8 * 1024 * 1024));

  hello<<<2, 2>>>();
  CHECK(cudaGetLastError());          // catch launch errors
  CHECK(cudaDeviceSynchronize());     // flush device-side printf
  CHECK(cudaDeviceReset());           // final flush & clean exit
  return 0;
}


Writing hello.cu


In [ ]:
!nvcc -arch=sm_75 hello.cu -o hello
!./hello


Hello from block 0, thread 0
Hello from block 0, thread 1
Hello from block 1, thread 0
Hello from block 1, thread 1


In [ ]:
%%writefile softmax.cu
#include <stdio.h>
#include <cuda.h>

#include <cuda_runtime.h>
#include <math.h>

const int N = 1024*32*8;

const int threadsPerBlock = 256;
const int blocksPerGrid = 128;

#define CUDA_CHECK(ans)                        \
    {                                          \
        cudaAssert((ans), __FILE__, __LINE__); \
    }
inline void cudaAssert(cudaError_t code, const char *file, int line) {
    if (code != cudaSuccess) {
        fprintf(stderr, "CUDA error %s: %s at %s: %d\n",
                cudaGetErrorName(code), cudaGetErrorString(code),
                file, line);
        exit(code);
    }
}





__global__ void softmax(float *vec, float *res) {
  __shared__ float cache[threadsPerBlock]; //next level on the memory hierarchy, scope is per block so want the number of threads per
  float local_max = -INFINITY;
  float local_norm = 0.0f;

  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  int cacheIndex = threadIdx.x; //why? for each block, want one entry for each thread

  //online softmax
  for (int i = threadIdx.x; i < blocksPerGrid; i+= blockDim.x) {
    float x = vec[i];

    if (x > local_max) {
      local_norm *= expf(local_max - x);
      local_max = x;
    }
    local_norm += expf(x - local_max);
  }

  __syncthreads(); //i want to try implementing the syncthreads primitive using semaphores later
  //now we have thread level local_norms and local_maxes, the next step is to reduce them thru shared memory in logN time to get a globalmax

  cache[threadIdx.x] = local_max;
  __syncthreads();
  for (int stride = threadsPerBlock/2; stride != 0; stride = stride/2) {
    if (threadIdx.x < stride) {
      cache[threadIdx.x] = fmax(cache[threadIdx.x], cache[threadIdx.x + stride]);
    }

    __syncthreads();

  }
  //now we have a global max at cache[0], so we need to correct the local_norm
  float global_max = cache[0];
  local_norm *= expf(local_max - global_max);

  cache[threadIdx.x] = local_norm;

  for (int stride = threadsPerBlock/2; stride != 0; stride = stride/2) {
    if (threadIdx.x < stride) {
      cache[threadIdx.x] += cache[threadIdx.x + stride];
    }
    __syncthreads();

  }

  float global_norm = cache[0]; //now we have the global norm stored as well, can actually do the softmax computation now
  printf("%f", global_norm);

  for (int i = tid; i < N; i += blockDim.x) {
    res[i] = expf(vec[i] - global_max) / global_norm;

  }



}

int main() {
  dim3 block_size(threadsPerBlock);
  dim3 grid_size(blocksPerGrid);

  float *vec;
  float *res;


  vec = (float *)malloc(N*sizeof(float));
  res = (float *)malloc(N*sizeof(float));

  for (int i = 0; i < N; i++) {
    vec[i] = log(i);
    res[i] = 0;
  }

  cudaError_t err;

  float *dev_vec;
  float *dev_res;

  err = cudaMalloc((void**)&dev_vec, N*sizeof(float));
  if (err != cudaSuccess) {
    fprintf(stderr, "cudaMalloc failed for dev_vec: %s\n", cudaGetErrorString(err));
    return;
}

  err = cudaMalloc((void**)&dev_res, N*sizeof(float));

  if (err != cudaSuccess) {
    fprintf(stderr, "cudaMalloc failed for dev_res: %s\n", cudaGetErrorString(err));
    return;
}


  cudaMemcpy(dev_vec, vec, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaEvent_t start, stop;
  CUDA_CHECK(cudaEventCreate(&start));
  CUDA_CHECK(cudaEventCreate(&stop));
  float ms = 0.f;

  CUDA_CHECK(cudaEventRecord(start));

  softmax<<<grid_size, block_size>>>(dev_vec, dev_res);

  cudaMemcpy(res, dev_res, N*sizeof(float), cudaMemcpyDeviceToHost);


  CUDA_CHECK(cudaEventRecord(stop));
  CUDA_CHECK(cudaEventSynchronize(stop));
  CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
  printf(">> Kernel execution time: %f ms\n", ms);
  printf("Slice of res: %f, %f, %f", res[0], res[1], res[2]);


  CUDA_CHECK(cudaEventDestroy(start));
  CUDA_CHECK(cudaEventDestroy(stop));

}



Overwriting softmax.cu


In [ ]:
!nvcc -arch=sm_75 softmax.cu -o softmax
!./softmax


softmax.cu(99): warning #117-D: non-void function "main" should return a value
      return;
            ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

softmax.cu(106): warning #117-D: non-void function "main" should return a value
      return;
            ^

softmax.cu(99): warning #117-D: non-void function "main" should return a value
      return;
            ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

softmax.cu(106): warning #117-D: non-void function "main" should return a value
      return;
            ^

CUDA error cudaErrorIllegalAddress: an illegal memory access was encountered at softmax.cu: 124


In [ ]:
%%writefile dotprod.cu
#include <stdio.h>
#include <cuda.h>

#include <cuda_runtime.h>
#include <math.h>
#define imin(a,b) (a<b?a:b)



const int N = 1024;
const int threadsPerBlock = 256;

const int blocksPerGrid = imin(32, (N + threadsPerBlock - 1)/threadsPerBlock);




#define CUDA_CHECK(ans)                        \
    {                                          \
        cudaAssert((ans), __FILE__, __LINE__); \
    }
inline void cudaAssert(cudaError_t code, const char *file, int line) {
    if (code != cudaSuccess) {
        fprintf(stderr, "CUDA error %s: %s at %s: %d\n",
                cudaGetErrorName(code), cudaGetErrorString(code),
                file, line);
        exit(code);
    }
}


__global__ void dot(int* a, int *b, int *c) {

  __shared__ int cache[threadsPerBlock];

  int tid = threadIdx.x + blockDim.x * blockIdx.x;

  int temp =0;


  while (tid < N) {
    temp += a[tid]*b[tid];
    tid += blockDim.x*gridDim.x; //stride by the number of total threads to get to the next access point for our curr thread
  }
  cache[threadIdx.x] = temp; //now put temp val in block-wide cache so we can then reduce per block
  __syncthreads();

  for (int stride = threadsPerBlock/2; stride > 0; stride /= 2) {
    if (threadIdx.x < stride) {
      cache[threadIdx.x] += cache[threadIdx.x + stride];
    }
    __syncthreads();
  }
  //now at cache[0] is our blockreduced sum, we can load it into int *c

  c[blockIdx.x] = cache[0];

}

int main() {
  dim3 block_size(threadsPerBlock);
  dim3 grid_size(blocksPerGrid);

  cudaError_t err;


  int *a;
  int *b;
  int *c;
  int *dev_a;
  int *dev_b;
  int *dev_c;


  a = (int *)malloc(N*sizeof(int));
  b = (int *)malloc(N*sizeof(int));
  c = (int *)malloc(blocksPerGrid*sizeof(int));

  for (int i = 0; i < N; i++) {
    a[i] = i;
    b[i] = i;
  }

  err = cudaMalloc((void**)&dev_a, N*sizeof(int));
  err = cudaMalloc((void**)&dev_b, N*sizeof(int));
  err = cudaMalloc((void**)&dev_c, blocksPerGrid*sizeof(int));

  cudaMemcpy(dev_a, a, N*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(dev_b, b, N*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(dev_c, c, blocksPerGrid*sizeof(int), cudaMemcpyHostToDevice);

  cudaEvent_t start, stop;
  CUDA_CHECK(cudaEventCreate(&start));
  CUDA_CHECK(cudaEventCreate(&stop));
  float ms = 0.f;

  CUDA_CHECK(cudaEventRecord(start));


  dot<<<grid_size, block_size>>>(dev_a, dev_b, dev_c);

  CUDA_CHECK(cudaEventRecord(stop));
  CUDA_CHECK(cudaEventSynchronize(stop));
  CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
  printf(">> Kernel execution time: %f ms\n", ms);

  CUDA_CHECK(cudaEventDestroy(start));
  CUDA_CHECK(cudaEventDestroy(stop));

  cudaMemcpy(c, dev_c, blocksPerGrid*sizeof(int), cudaMemcpyDeviceToHost);
  int sum = 0;

  for (int i=0; i < blocksPerGrid; i++) {
    sum+= c[i];
  }

  printf("dotprod is %i", sum);
  free(a);
  free(b);
  free(c);
  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);


}

Overwriting dotprod.cu


In [ ]:
!nvcc -arch=sm_75 dotprod.cu -o dot
!./dot


dotprod.cu(64): warning #550-D: variable "err" was set but never used
    cudaError_t err;
                ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

>> Kernel execution time: 0.165280 ms
dotprod is 357389824